In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###  Importing training and test dataset

In [2]:
df_whole = pd.read_csv("/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/train.csv")

In [ ]:
test_kaggle = pd.read_csv("/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/test.csv")

###  Training model on 50% of data

In [24]:
# splitting data randomly (not using whole data for now)
train, drop = train_test_split(df_whole, test_size=0.8)

In [25]:
train.shape
drop.shape

(11084771, 8)

(44339085, 8)

In [26]:
#removing index
train = train.reset_index()

train =  train.drop('index', 1)

### Creating variables out of pickup_datetime

In [27]:
# creating year
train['year'] = train.pickup_datetime.apply(lambda x: x[0:4])

#creating month
train['month'] = train.pickup_datetime.apply(lambda x: x[5:7])

#year_month
train['year_month'] = train.pickup_datetime.apply(lambda x: x[0:7])

#Day_of_month
train['day_of_month'] = train.pickup_datetime.apply(lambda x: x[8:10])

#hour
train['hour'] = train.time.apply(lambda x: x[0:2])

#minute
train['minute'] = train.time.apply(lambda x: x[3:5])

#date
train['date'] = train.pickup_datetime.apply(lambda x: x[0:10])

#time
train['time'] = train.pickup_datetime.apply(lambda x: x[11:16])

#time_zone --just to make sure all are same
# tested they all are same, so no need of creating this field
#train['time_zone'] = train.pickup_datetime.apply(lambda x: x[-3:])


In [28]:
# test: to check values while slicing
# train.pickup_datetime[13][-3:]

In [29]:
# converting date column into date format
train['date'] = pd.to_datetime(train['date'])

# creating day of the week column
train['day_of_week'] = train['date'].dt.day_name()

In [30]:
# checking split of rides according to day of the week
train['day_of_week'].value_counts()

Friday       1705126
Saturday     1680301
Thursday     1661795
Wednesday    1610603
Tuesday      1552886
Sunday       1453865
Monday       1420195
Name: day_of_week, dtype: int64

In [50]:
# creating weekend/weekday column
# weekday = 0 & weekday = 1
train['weekend_weekday'] = train.day_of_week.apply(lambda x: '1' if x in ('Friday', 'Saturday', 'Sunday') else '0')

In [32]:
# checking timezone types
# train['time_zone'].value_counts()

In [33]:
#only one time zone detected,
# dropping timezone column
# train = train.drop('time_zone',1)

In [34]:
#dropping datetime column
train = train.drop('pickup_datetime', 1)

In [35]:
# converting data type of columns
train['passenger_count'] = train['passenger_count'].astype('category')
train['year'] = train['year'].astype('category')
train['month'] = train['month'].astype('category')
#train['year_month'] = train['year_month'].astype('category')
train['day_of_week'] = train['day_of_week'].astype('category')
train['weekend_weekday'] = train['weekend_weekday'].astype('category')

In [36]:
train.dtypes

key                          object
fare_amount                 float64
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count            category
year                       category
month                      category
year_month                   object
day_of_month                 object
date                 datetime64[ns]
time                         object
day_of_week                category
weekend_weekday            category
dtype: object

In [95]:
#saving train file
train.to_csv('train_2.csv', index=False )

####  getting address from lat long

In [93]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,year_month,day_of_month,date,time,day_of_week,weekend_weekday
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,03,2015-03,10,2015-03-10,07:45,Tuesday,0
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,2011-12,17,2011-12-17,00:49,Saturday,1
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,09,2010-09,16,2010-09-16,00:24,Thursday,0
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,06,2012-06,13,2012-06-13,18:03,Wednesday,0
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,02,2015-02,04,2015-02-04,07:31,Wednesday,0


In [ ]:
!pip install pygeocoder

In [ ]:
# package to get address from lat long
from pygeocoder import Geocoder

In [ ]:
results = Geocoder.reverse_geocode(train['dropoff_latitude'][0], train['dropoff_longitude'][0])

In [43]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,year_month,day_of_month,date,time,day_of_week,weekend_weekday
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,03,2015-03,10,2015-03-10,07:45,Tuesday,weekday
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,2011-12,17,2011-12-17,00:49,Saturday,weekend
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,09,2010-09,16,2010-09-16,00:24,Thursday,weekday
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,06,2012-06,13,2012-06-13,18:03,Wednesday,weekday
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,02,2015-02,04,2015-02-04,07:31,Wednesday,weekday


In [52]:
train.isnull().values.any()

False

In [39]:
train = train.dropna() 

In [ ]:
#Splitting into test and trin
# splitting data randomly (not using whole data for now, taking very small chunk)
#train, test = train_test_split(train, test_size=0.25)

In [ ]:
#train.shape
#test.shape

In [ ]:
#removing key and pickup_datetime
train = train.drop('key', 1)

In [42]:
train = train.drop('pickup_datetime', 1)

KeyError: "['pickup_datetime'] not found in axis"

In [53]:
# splitting data randomly (not using whole data for now)
train_model, test_model = train_test_split(train, test_size=0.1)

In [54]:
# Split the data into training/testing sets
train_label = train_model.fare_amount
train_data = train_model.drop('fare_amount', 1)

In [55]:
test_label = test_model.fare_amount
test_data = test_model.drop('fare_amount', 1)

####  creating test model and predicting on base of it

In [56]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [57]:
# Train the model using the training sets
regr.fit(train_data, train_label)

ValueError: could not convert string to float: 'Sunday'

In [ ]:
# Make predictions using the testing set
test_label_pred = regr.predict(test_data)

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(test_label, test_label_pred))

In [ ]:
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(test_label, test_label_pred))

####  predicting on given test data

In [ ]:
test_kaggle.head()

In [ ]:
test_kaggle.key[8][14:21]

In [ ]:
# ?test_kaggle.key = test_kaggle.key.apply(lambda x: x[14:21])

In [ ]:
# kaggle_key = test_kaggle.key
test_kaggle = test_kaggle.drop('key', 1)
test_kaggle = test_kaggle.drop('pickup_datetime', 1)

In [ ]:
# Make predictions using the testing set
test_kagle_label_pred = regr.predict(test_kaggle)

In [ ]:
#checking values
test_kagle_label_pred[1:5]

In [ ]:
#rounding of to two decimal
import numpy as np
test_kagle_label_pred = np.round(test_kagle_label_pred, decimals=2)

In [ ]:
#checking values
test_kagle_label_pred[1:5]

In [ ]:
submission = pd.DataFrame()

In [ ]:
submission.key = pd.DataFrame(test_kaggle.key)
submission.fare_amount = pd.DataFrame(test_kagle_label_pred)

In [ ]:
col_names =  ['key', 'fare_amount']
submission  = pd.DataFrame(columns = col_names)


In [ ]:
submission.head()

In [ ]:
submission.key = (test_kaggle.key)
submission.fare_amount = test_kagle_label_pred

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_2.csv', index=False )

#### Encoding string

In [79]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [90]:
le.fit(train.day_of_week)

LabelEncoder()

In [91]:
list(le.classes_)

['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

In [92]:
le.transform(train.day_of_week) 

array([5, 2, 4, ..., 3, 0, 5], dtype=int64)

#### hot coding

In [73]:
from sklearn.preprocessing import OneHotEncoder

In [74]:
enc = OneHotEncoder(handle_unknown='ignore')

In [85]:
X = [['Male', 1], ['Female', 3], ['Female', 2]]

In [86]:
enc.fit(X)

ValueError: could not convert string to float: 'Male'

In [78]:

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<... 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

SyntaxError: invalid syntax (<ipython-input-78-e9e2d924eb62>, line 3)

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [66]:
X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

In [67]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                              random_state=0)

In [72]:
clf.fit(train_data, train_label)

ValueError: could not convert string to float: 'Sunday'

In [ ]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
print(clf.feature_importances_)

In [ ]:
print(clf.predict([[0, 0, 0, 0]]))